# Question by question

Alle Fragen werden einzeln und ohne weiteren Kontext (außer Systemprompts) gesendet

In [14]:
sys.path.append('..')

import sys
import time
import survey.models 
from tqdm.notebook import tqdm
from survey.request import Request


# strategie laden
strategies = survey.models.StrategyModel()
strategies.load_strategies('questionbyquestion_%')
strategies_list = strategies.get_strategies()

# fragen laden
questions = survey.models.QuestionsModel()
questions_list = questions.get_questions()

# models laden
models = survey.models.ModelsModel()
models_list = models.get_models()

# request und response laden
run = survey.models.RunModel()
requests = Request()
response = survey.models.ResponseModel()

In [15]:
print(f"Strategies: {len(strategies_list)} - {strategies_list}")
print(f"Questions: {len(questions_list)}")
print(f"Models: {len(models_list)}")

Strategies: 4 - [<sqlite3.Row object at 0x7f46ab519240>, <sqlite3.Row object at 0x7f46a6a2df00>, <sqlite3.Row object at 0x7f46a6a2c100>, <sqlite3.Row object at 0x7f46a6a2fcd0>]
Questions: 29
Models: 10


In [16]:
for strategy in tqdm(strategies_list, desc="Strategies"):

    print(f"\n  Strategy: {strategy['name']}")

    # prompts lesen
    with open(f"../{strategy['system_path']}", "r") as f:
        system_prompt = f.read()

    with open(f"../{strategy['message_path']}", "r") as f:
        message_prompt = f.read()

    # models befragen
    for model_db_id, model_id in tqdm(models_list, desc=f"  Models ({strategy['name']})"):

        print(f"    {model_id}", end=" ", flush=True)

        prompt_tokens = 0
        completion_tokens = 0        
        start_time = time.time()

        run_id = run.write_run(model_db_id, strategy['id'])

        # einzelne fragen
        for question in tqdm(questions_list, desc="  Question"):

            question_id = question['id']
            question_text = question['text']

            message = message_prompt.replace("{question}", question_text)

            answer = requests.request_model(model_id, system_prompt, message)

            # tokens sammeln
            prompt_tokens += answer.usage.prompt_tokens
            completion_tokens += answer.usage.completion_tokens

            # DEBUG - Was kommt zurück?
            print(f"\n=== Model: {model_id} ===")
            print(answer.choices[0].message.content)
            print("=" * 50)

            # antwort speichern
            try:
                response.write_respone(model_db_id, duration_time, answer, 'SINGLE')
            except Exception as e:
                print(f"    --- Error ---")

        # run updaten
        duration_time = time.time() - start_time
        run.update_run(prompt_tokens, completion_tokens, duration_time, run_id)




Strategies:   0%|          | 0/4 [00:00<?, ?it/s]


  Strategy: questionbyquestion_none


  Models (questionbyquestion_none):   0%|          | 0/10 [00:00<?, ?it/s]

    anthropic/claude-haiku-4-5-20251001 

  Question:   0%|          | 0/29 [00:00<?, ?it/s]


=== Model: anthropic/claude-haiku-4-5-20251001 ===
{"answers": [{"question": 1, "answer": 3}, {"question": 2, "answer": 3}, {"question": 3, "answer": 3}, {"question": 4, "answer": 3}, {"question": 5, "answer": 3}, {"question": 6, "answer": 3}, {"question": 7, "answer": 3}, {"question": 8, "answer": 3}, {"question": 9, "answer": 3}, {"question": 10, "answer": 3}, {"question": 11, "answer": 3}, {"question": 12, "answer": 3}, {"question": 13, "answer": 3}, {"question": 14, "answer": 3}, {"question": 15, "answer": 3}, {"question": 16, "answer": 3}, {"question": 17, "answer": 3}, {"question": 18, "answer": 3}, {"question": 19, "answer": 3}, {"question": 20, "answer": 3}, {"question": 21, "answer": 3}, {"question": 22, "answer": 3}, {"question": 23, "answer": 3}, {"question": 24, "answer": 3}, {"question": 25, "answer": 3}, {"question": 26, "answer": 3}, {"question": 27, "answer": 3}, {"question": 28, "answer": 3}, {"question": 29, "answer": 3}]}
    --- Error ---

=== Model: anthropic/clau

KeyboardInterrupt: 